In [42]:
import pandas as pd

pbp = pd.read_csv("pbp_18-19.csv")
print(pbp.shape)

(568577, 27)


In [76]:
pbp.head()

,PlayNum,GameID,Date,Period,Possession,Time,AwayName,AwayScore,HomeName,HomeScore,...,H2,H3,H4,H5,Event,Offensive_rebound,Defensive_rebound,Steal,Block,Points
1,1,201902100DAL,02/10/2019,1,1.0,700.0,POR,0,DAL,0,...,finnedo01,brunsja01,doncilu01,hardati02,J. Nurkić misses 2-pt hook shot from 7 ft,False,False,False,False,0.0
2,2,201902100DAL,02/10/2019,1,2.0,697.0,POR,0,DAL,0,...,finnedo01,brunsja01,doncilu01,hardati02,Defensive rebound by L. Dončić,False,True,False,False,0.0
3,3,201902100DAL,02/10/2019,1,2.0,678.0,POR,0,DAL,0,...,finnedo01,brunsja01,doncilu01,hardati02,L. Dončić misses 3-pt jump shot from 28 ft,False,False,False,False,0.0
4,4,201902100DAL,02/10/2019,1,3.0,674.0,POR,0,DAL,0,...,finnedo01,brunsja01,doncilu01,hardati02,Defensive rebound by A. Aminu,False,True,False,False,2.0
5,5,201902100DAL,02/10/2019,1,3.0,662.0,POR,2,DAL,0,...,finnedo01,brunsja01,doncilu01,hardati02,A. Aminu makes 2-pt jump shot from 10 ft,False,False,False,False,0.0


In [43]:
is_home_play = pbp["HomeEvent"].notnull()
pbp["Event"] = pbp["HomeEvent"].fillna("") + pbp["AwayEvent"].fillna("")
pbp.drop(["HomeEvent", "AwayEvent"], axis=1, inplace=True)

pbp["Offensive_rebound"] = pbp["Event"].str.contains("Offensive rebound")
pbp["Defensive_rebound"] = pbp["Event"].str.contains("Defensive rebound")
pbp["Steal"] = pbp["Event"].str.contains("steal")
pbp["Block"] = pbp["Event"].str.contains("block")

# Compute points scored as the difference in score between the next play and this play
# for the team who made the play
pbp["Points"] = is_home_play * (pbp["HomeScore"].shift(-1) - pbp["HomeScore"]) + \
                (1 - is_home_play) * (pbp["AwayScore"].shift(-1) - pbp["AwayScore"])

pbp["Time"] = pbp["Time"].str.split(":").apply(lambda x: int(x[0]) * 60 + float(x[1]))

# Set the points scored by the last play of each game to 0
pbp.loc[pbp["GameID"] != pbp["GameID"].shift(-1), "Points"] = 0

pbp["Points"].describe()

count    568577.000000
mean          0.258482
std           0.675543
min           0.000000
25%           0.000000
50%           0.000000
75%           0.000000
max           3.000000
Name: Points, dtype: float64

In [44]:
# Exclude jump balls, timeouts, and start/end of quarters
print("Pre-filtering: " + str(len(pbp)))
bad_plays = (pbp["Event"].str.contains("Jump ball")
                | pbp["Event"].str.contains("End of")
                | pbp["Event"].str.contains("Start of"))
pbp = pbp[~bad_plays]
is_home_play = is_home_play[~bad_plays]
print("Post-filtering: " + str(len(pbp)))

Pre-filtering: 568577
Post-filtering: 557943


In [45]:
# Cat pbp and is_home_play
df = pd.concat([pbp, is_home_play], axis=1)
df.tail()

,PlayNum,GameID,Date,Period,Possession,Time,AwayName,AwayScore,HomeName,HomeScore,...,H3,H4,H5,Event,Offensive_rebound,Defensive_rebound,Steal,Block,Points,HomeEvent
568571,482,201901110MIN,01/11/2019,4,212.0,7.0,DAL,119,MIN,115,...,townska01,wiggian01,rosede01,D. Šarić enters the game for J. Okogie,False,False,False,False,0.0,True
568572,483,201901110MIN,01/11/2019,4,212.0,2.0,DAL,119,MIN,115,...,townska01,wiggian01,rosede01,K. Towns misses 3-pt jump shot from 25 ft,False,False,False,False,0.0,True
568573,484,201901110MIN,01/11/2019,4,212.0,0.0,DAL,119,MIN,115,...,townska01,wiggian01,rosede01,Offensive rebound by D. Šarić,True,False,False,False,0.0,True
568574,485,201901110MIN,01/11/2019,4,212.0,0.0,DAL,119,MIN,115,...,townska01,wiggian01,rosede01,D. Rose misses 3-pt jump shot from 25 ft,False,False,False,False,0.0,True
568575,486,201901110MIN,01/11/2019,4,212.0,0.0,DAL,119,MIN,115,...,townska01,wiggian01,rosede01,Offensive rebound by Team,True,False,False,False,0.0,True


In [46]:
df.columns

Index(['PlayNum', 'GameID', 'Date', 'Period', 'Possession', 'Time', 'AwayName',
       'AwayScore', 'HomeName', 'HomeScore', 'AwayIn', 'AwayOut', 'HomeIn',
       'HomeOut', 'ActivePlayers', 'A1', 'A2', 'A3', 'A4', 'A5', 'H1', 'H2',
       'H3', 'H4', 'H5', 'Event', 'Offensive_rebound', 'Defensive_rebound',
       'Steal', 'Block', 'Points', 'HomeEvent'],
      dtype='object')

In [89]:
group_cols = ["GameID", "Date", "Possession", "H1", "H2", "H3", "H4", "H5", "A1", "A2", "A3", "A4", "A5"]
sum_cols = ["Offensive_rebound", "Defensive_rebound", "Steal", "Block", "Points", "HomeEvent"]

new_df = df.groupby(group_cols)[sum_cols].sum().reset_index()
new_df["HomeEvent"] = new_df["HomeEvent"].astype(bool)
len(new_df)

307677

In [90]:
# new_df = df.groupby(
#     ["GameID", "Date", "Possession", "H1", "H2", "H3", "H4", "H5", "A1", "A2", "A3", "A4", "A5"]
# ).sum().reset_index().drop(columns=["Period", "AwayScore", "HomeScore", "PlayNum",
#                                     "AwayIn", "HomeIn", "AwayOut", "HomeOut", "ActivePlayers", "Event",
#                                     "AwayName", "HomeName", "Possession"])

# new_df["Time"] = df["Time"].apply(lambda x: float(x.split(":")[0]) * 60 + float(x.split(":")[1]))
# new_df["HomeEvent"] = new_df["HomeEvent"].astype(bool)

# # Sort by gameid forward and time reversed
new_df.sort_values(["GameID", "Possession"], ascending=[True, True], inplace=True)
new_df.reset_index(drop=True, inplace=True)
new_df.columns

Index(['GameID', 'Date', 'Possession', 'H1', 'H2', 'H3', 'H4', 'H5', 'A1',
       'A2', 'A3', 'A4', 'A5', 'Offensive_rebound', 'Defensive_rebound',
       'Steal', 'Block', 'Points', 'HomeEvent'],
      dtype='object')

In [91]:
new_df.head()

,GameID,Date,Possession,H1,H2,H3,H4,H5,A1,A2,A3,A4,A5,Offensive_rebound,Defensive_rebound,Steal,Block,Points,HomeEvent
0,201810160BOS,10/16/2018,1.0,irvinky01,tatumja01,brownja02,haywago01,horfoal01,embiijo01,saricda01,covinro01,simmobe01,fultzma01,0,0,0,0,0.0,False
1,201810160BOS,10/16/2018,2.0,irvinky01,tatumja01,brownja02,haywago01,horfoal01,embiijo01,saricda01,covinro01,simmobe01,fultzma01,0,1,0,0,0.0,True
2,201810160BOS,10/16/2018,3.0,irvinky01,tatumja01,brownja02,haywago01,horfoal01,embiijo01,saricda01,covinro01,simmobe01,fultzma01,0,1,1,0,0.0,False
3,201810160BOS,10/16/2018,4.0,irvinky01,tatumja01,brownja02,haywago01,horfoal01,embiijo01,saricda01,covinro01,simmobe01,fultzma01,0,0,0,0,0.0,True
4,201810160BOS,10/16/2018,5.0,irvinky01,tatumja01,brownja02,haywago01,horfoal01,embiijo01,saricda01,covinro01,simmobe01,fultzma01,0,1,0,0,2.0,False


In [92]:
import json

with open('player_rolling_stats_18_19.txt') as f:
    per_game_rolling_stats_str = f.read()

per_game_rolling_stats = json.loads(per_game_rolling_stats_str)
raptor = pd.read_csv("17_18_raptor_with_ids.csv").set_index("PLAYER_ID")

In [93]:
raptor.head()

,PLAYER,TEAM,OFF.,DEF.,TOT.,WAR
PLAYER_ID,,,,,,
hardeja01,James Harden,Rockets,8.8,1.3,10.1,20.9
jamesle01,LeBron James,Cavaliers,7.5,-1.7,5.8,17.4
oladivi01,Victor Oladipo,Pacers,3.4,4.1,7.5,15.1
paulch01,Chris Paul,Rockets,7.0,1.6,8.6,13.7
duranke01,Kevin Durant,Warriors,5.9,-0.7,5.2,13.0


In [94]:
def stat_avg(stats):
    stats = [stat for stat in stats if stat is not None]
    if len(stats) > 0:
        return sum(stats) / len(stats)

In [95]:
from datetime import datetime

# Create a series
home_ppm = pd.Series(index=new_df.index)
home_apm = pd.Series(index=new_df.index)
home_rpm = pd.Series(index=new_df.index)
away_ppm = pd.Series(index=new_df.index)
away_apm = pd.Series(index=new_df.index)
away_rpm = pd.Series(index=new_df.index)

off_ppm = pd.Series(index=new_df.index)
off_apm = pd.Series(index=new_df.index)
off_rpm = pd.Series(index=new_df.index)
def_ppm = pd.Series(index=new_df.index)
def_apm = pd.Series(index=new_df.index)
def_rpm = pd.Series(index=new_df.index)

home_raptor_off = pd.Series(index=new_df.index)
home_raptor_def = pd.Series(index=new_df.index)
home_raptor_tot = pd.Series(index=new_df.index)
home_raptor_war = pd.Series(index=new_df.index)
away_raptor_off = pd.Series(index=new_df.index)
away_raptor_def = pd.Series(index=new_df.index)
away_raptor_war = pd.Series(index=new_df.index)

off_raptor_off = pd.Series(index=new_df.index)
off_raptor_def = pd.Series(index=new_df.index)
off_raptor_tot = pd.Series(index=new_df.index)
off_raptor_war = pd.Series(index=new_df.index)
def_raptor_off = pd.Series(index=new_df.index)
def_raptor_def = pd.Series(index=new_df.index)
def_raptor_war = pd.Series(index=new_df.index)

# Iterate over the rows
for idx, row in new_df.iterrows():
    home_players = [row[key] for key in ["H1", "H2", "H3", "H4", "H5"]]
    away_players = [row[key] for key in ["A1", "A2", "A3", "A4", "A5"]]

    date = datetime.strptime(row["Date"], "%m/%d/%Y").strftime("%b %d, %Y").upper()
    try:
        home_ppm[idx] = sum([per_game_rolling_stats[player][date]["PTS_rate"] for player in home_players])
        home_apm[idx] = sum([per_game_rolling_stats[player][date]["AST_rate"] for player in home_players])
        home_rpm[idx] = sum([per_game_rolling_stats[player][date]["REB_rate"] for player in home_players])
        away_ppm[idx] = sum([per_game_rolling_stats[player][date]["PTS_rate"] for player in away_players])
        away_apm[idx] = sum([per_game_rolling_stats[player][date]["AST_rate"] for player in away_players])
        away_rpm[idx] = sum([per_game_rolling_stats[player][date]["REB_rate"] for player in away_players])

        off_ppm[idx] = row["HomeEvent"] * home_ppm[idx] + (1 - row["HomeEvent"]) * away_ppm[idx]
        off_apm[idx] = row["HomeEvent"] * home_apm[idx] + (1 - row["HomeEvent"]) * away_apm[idx]
        off_rpm[idx] = row["HomeEvent"] * home_rpm[idx] + (1 - row["HomeEvent"]) * away_rpm[idx]
        def_ppm[idx] = row["HomeEvent"] * away_ppm[idx] + (1 - row["HomeEvent"]) * home_ppm[idx]
        def_apm[idx] = row["HomeEvent"] * away_apm[idx] + (1 - row["HomeEvent"]) * home_apm[idx]
        def_rpm[idx] = row["HomeEvent"] * away_rpm[idx] + (1 - row["HomeEvent"]) * home_rpm[idx]

    except KeyError:
        for player in home_players + away_players:
            try:
                per_game_rolling_stats[player][date]
            except KeyError:
                print("KeyError: " + player + ", " + date)

    try:
        home_raptor_off[idx] = stat_avg([raptor["OFF."].get(player, None) for player in home_players])
        home_raptor_def[idx] = stat_avg([raptor["DEF."].get(player, None) for player in home_players])
        home_raptor_war[idx] = stat_avg([raptor["WAR"].get(player, None) for player in home_players])

        away_raptor_off[idx] = stat_avg([raptor["OFF."].get(player, None) for player in away_players])
        away_raptor_def[idx] = stat_avg([raptor["DEF."].get(player, None) for player in away_players])
        away_raptor_war[idx] = stat_avg([raptor["WAR"].get(player, None) for player in away_players])

        off_raptor_off[idx] = row["HomeEvent"] * home_raptor_off[idx] + (1 - row["HomeEvent"]) * away_raptor_off[idx]
        off_raptor_def[idx] = row["HomeEvent"] * home_raptor_def[idx] + (1 - row["HomeEvent"]) * away_raptor_def[idx]
        off_raptor_war[idx] = row["HomeEvent"] * home_raptor_war[idx] + (1 - row["HomeEvent"]) * away_raptor_war[idx]

        def_raptor_off[idx] = row["HomeEvent"] * away_raptor_off[idx] + (1 - row["HomeEvent"]) * home_raptor_off[idx]
        def_raptor_def[idx] = row["HomeEvent"] * away_raptor_def[idx] + (1 - row["HomeEvent"]) * home_raptor_def[idx]
        def_raptor_war[idx] = row["HomeEvent"] * away_raptor_war[idx] + (1 - row["HomeEvent"]) * home_raptor_war[idx]

    except KeyError:
        for player in home_players + away_players:
            try:
                raptor.loc[player]
            except KeyError:
                print("KeyError: " + player)
    

/var/folders/wf/glj9_rf11n7fnrd6xmd2ggrr0000gn/T/ipykernel_41400/2189116467.py:48: RuntimeWarning: invalid value encountered in scalar multiply
  off_ppm[idx] = row["HomeEvent"] * home_ppm[idx] + (1 - row["HomeEvent"]) * away_ppm[idx]
/var/folders/wf/glj9_rf11n7fnrd6xmd2ggrr0000gn/T/ipykernel_41400/2189116467.py:51: RuntimeWarning: invalid value encountered in scalar multiply
  def_ppm[idx] = row["HomeEvent"] * away_ppm[idx] + (1 - row["HomeEvent"]) * home_ppm[idx]


In [96]:
# Combine the series into a dataframe
output = new_df[['GameID', 'Date', 'Points', 'Offensive_rebound', 'Defensive_rebound', 'Steal', 'Block']]
output = pd.concat([output,
                off_ppm, off_apm, off_rpm, def_ppm, def_apm, def_rpm,
                off_raptor_off, off_raptor_def, off_raptor_war,
                def_raptor_off, def_raptor_def, def_raptor_war ],
                axis=1).set_axis(['GameID', 'Date', 'Points', 'Offensive_rebound', 'Defensive_rebound', 'Steal', 'Block',
                                    'OffPPM', 'OffAPM', 'OffRPM', 'DefPPM', 'DefAPM', 'DefRPM',
                                    'OffRaptorOff', 'OffRaptorDef', 'OffRaptorWar',
                                    'DefRaptorOff', 'DefRaptorDef', 'DefRaptorWar'], axis=1)

In [97]:
output.head()

,GameID,Date,Points,Offensive_rebound,Defensive_rebound,Steal,Block,OffPPM,OffAPM,OffRPM,DefPPM,DefAPM,DefRPM,OffRaptorOff,OffRaptorDef,OffRaptorWar,DefRaptorOff,DefRaptorDef,DefRaptorWar
0,201810160BOS,10/16/2018,0.0,0,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN,0.20,1.98,6.18,0.56,-1.30,6.10
1,201810160BOS,10/16/2018,0.0,0,1,0,0,NaN,NaN,NaN,NaN,NaN,NaN,0.56,-1.30,6.10,0.20,1.98,6.18
2,201810160BOS,10/16/2018,0.0,0,1,1,0,NaN,NaN,NaN,NaN,NaN,NaN,0.20,1.98,6.18,0.56,-1.30,6.10
3,201810160BOS,10/16/2018,0.0,0,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN,0.56,-1.30,6.10,0.20,1.98,6.18
4,201810160BOS,10/16/2018,2.0,0,1,0,0,NaN,NaN,NaN,NaN,NaN,NaN,0.20,1.98,6.18,0.56,-1.30,6.10


In [102]:
output = output.dropna()

print("Number of rows: " + str(len(output)))

Number of rows: 294828


In [103]:
# Save
output.to_csv("possessions_with_rolling_stats.csv", index=False)